<a href="https://colab.research.google.com/github/plthomps/CIS-3902-Data-Mining/blob/main/Data_Cleaning_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning in Python — A Hands-On Tutorial and Exercise prepared by Dr. Pamela Thompson

**Course:** Intro to Data Mining  
**Your Name:** Add your name
**Dataset:** Titanic Passengers (loaded from a public URL, no file downloads needed!)  

---

## Why does data cleaning matter?

Real-world data is *messy*. Before we can build any model or draw any conclusions, we need to:

1. **Understand** the data (shape, types, distributions)
2. **Find problems** (missing values, duplicates, outliers, wrong types)
3. **Fix them** using appropriate strategies

In practice, data scientists spend **60–80 %** of their time on data cleaning and preparation. This notebook walks through the most common cleaning steps using the famous Titanic dataset.

> **Tip:** Run each cell one at a time and read the output before moving on. The goal is to understand *why* we do each step, not just *how*.

---
## 1. Load the Data & Take a First Look

We always start by loading the data and getting a quick overview. The key questions are:
- How many rows and columns?
- What do the first few rows look like?
- What are the column data types?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the full Titanic dataset from a public URL
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)

# Quick peek: shape tells us (rows, columns)
print(f'Dataset shape: {df.shape[0]} rows x {df.shape[1]} columns')

In [ ]:
# .head() shows the first 5 rows — always a good first step
df.head()

### What do the columns mean?

| Column | Description |
|--------|-------------|
| PassengerId | Unique ID for each passenger |
| Survived | 0 = No, 1 = Yes (this is what we'd want to predict) |
| Pclass | Ticket class: 1 = 1st, 2 = 2nd, 3 = 3rd |
| Name | Passenger name |
| Sex | male / female |
| Age | Age in years |
| SibSp | # of siblings / spouses aboard |
| Parch | # of parents / children aboard |
| Ticket | Ticket number |
| Fare | Passenger fare |
| Cabin | Cabin number |
| Embarked | Port of embarkation: C = Cherbourg, Q = Queenstown, S = Southampton |

---
## 2. Understand Data Types & Structure

`.info()` is one of the most useful commands in pandas. It tells us:
- The data type of each column (int, float, object)
- How many **non-null** values each column has (helps spot missing data!)
- Memory usage

In [ ]:
df.info()

**What to notice above:**
- `Age` has only 714 non-null values out of 891 → missing data!
- `Cabin` has only 204 non-null values → a lot of missing data!
- `Embarked` has 889 non-null values → just 2 missing
- Columns like `Name`, `Sex`, `Ticket` are `object` (text) type

### What is the `object` data type?

In pandas, **`object`** is the default type for columns that contain **text (strings)**. When you see `object` in the `.info()` output, it almost always means that column holds words/labels rather than numbers.

**Why does this matter?** Most machine learning algorithms can **only work with numbers**. They can't do math on words like `"male"` or `"Southampton"`. So before we can use text columns in a model, we need to **convert them to numbers** — a process called **encoding**. We'll do this in Section 6.

Here are the common pandas data types you'll see:

| Dtype | What it means | Example values |
|-------|--------------|----------------|
| `int64` | Whole numbers | 1, 2, 3 |
| `float64` | Decimal numbers | 7.25, 29.5 |
| `object` | Text / strings | "male", "S", "Braund, Mr. Owen" |
| `bool` | True/False | True, False |

### Separate categorical vs. numerical columns

This is important because different column types need different cleaning strategies.
- **Numerical columns** → check for outliers, scale, impute with mean/median
- **Categorical (`object`) columns** → check unique values, encode to numbers, impute with mode

In [ ]:
# Identify categorical columns (dtype == 'object' means text/string)
cat_cols = [col for col in df.columns if df[col].dtype == 'object']

# Identify numerical columns
num_cols = [col for col in df.columns if df[col].dtype != 'object']

print('Categorical columns:', cat_cols)
print('Numerical columns: ', num_cols)

In [ ]:
# For categorical columns, check how many unique values each has
# This helps us decide which ones are useful for analysis
df[cat_cols].nunique()

**Observation:** `Name` (891 unique) and `Ticket` (681 unique) have too many unique values to be useful as categorical features. We'll drop them later.

---
## 3. Descriptive Statistics

`.describe()` gives us summary statistics for numerical columns. Look for:
- Big differences between **mean** and **median (50%)** indicates possible skew or outliers
- **min/max** values that seem unreasonable
- The **count** row (does it match the total number of rows?)

In [ ]:
df.describe()

**Things to notice:**
- `Age` count is 714, not 891 which confirms missing values
- `Age` mean (29.7) and median/50% (28.0) are close and roughly symmetric
- `Fare` mean (32.2) is much higher than median (14.5) and right-skewed (some very expensive tickets)
- `Fare` min is 0.0 meaning some passengers paid nothing? Worth investigating

---
## 4. Check for Duplicate Rows

Duplicate rows can bias our analysis. We check for them and remove if needed.

In [ ]:
# .duplicated() returns True for each row that is a duplicate of an earlier row
num_duplicates = df.duplicated().sum()
print(f'Number of duplicate rows: {num_duplicates}')

Good news — no duplicates in this dataset. If there were, we'd remove them with:
```python
df = df.drop_duplicates()
```

---
## 5. Handle Missing Values

This is often the biggest part of data cleaning. We have three main strategies:

| Strategy | When to use |
|----------|------------|
| **Drop the column** | When most values are missing (e.g., > 50%) |
| **Drop the rows** | When very few rows are affected |
| **Fill (impute) the values** | When a moderate amount is missing and we want to keep the data |

Let's first see exactly how much is missing in each column.

In [ ]:
# Calculate the percentage of missing values for each column
missing_pct = (df.isnull().sum() / len(df)) * 100
missing_pct = missing_pct.round(2)

print('Percentage of missing values per column:')
print(missing_pct[missing_pct > 0])  # Only show columns with missing values

### Step 5a: Drop columns that won't be useful

- `Cabin` is 77% missing — too much to fill reliably. **Drop it.**
- `Name` and `Ticket` have too many unique values to be directly useful. **Drop them.**
- `PassengerId` is just a row identifier, not a real feature. **Drop it.**

In [ ]:
# Drop columns that aren't useful for analysis
cols_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']
df_clean = df.drop(columns=cols_to_drop)

print(f'Shape before: {df.shape}')
print(f'Shape after:  {df_clean.shape}')
df_clean.head()

### Step 5b: Handle `Embarked` (only 2 missing values)

Since only 2 rows out of 891 are missing, we can safely drop those rows.

In [ ]:
# Drop the 2 rows where Embarked is missing
print(f'Rows before: {len(df_clean)}')
df_clean = df_clean.dropna(subset=['Embarked'])
print(f'Rows after:  {len(df_clean)}')

### Step 5c: Handle `Age` (about 20% missing)

20% is too much to just drop as we would lose a lot of data. Instead, we'll **impute** (fill in) the missing values.

**Common imputation strategies:**
- **Mean** — good when data is roughly normally distributed
- **Median** — better when data is skewed or has outliers
- **Mode** — used for categorical data

Since Age's mean (29.7) and median (28.0) are close, either works here. We'll use the **median** since it's more robust to outliers.

In [ ]:
# Check the mean vs. median of Age before filling
print(f'Age mean:   {df_clean["Age"].mean():.1f}')
print(f'Age median: {df_clean["Age"].median():.1f}')

# Fill missing Age values with the median
df_clean['Age'] = df_clean['Age'].fillna(df_clean['Age'].median())

# Verify: no more missing values
print('\nMissing values remaining:')
print(df_clean.isnull().sum())

---
## 6. Encode Categorical Variables

Remember those `object` columns from Section 2? After dropping `Name`, `Ticket`, and `Cabin`, we still have two text columns left: **`Sex`** and **`Embarked`**. Machine learning algorithms need numbers, so we need to convert these text values into numerical form.

There are two common encoding methods:

| Method | How it works | Best for |
|--------|-------------|----------|
| **Label Encoding** | Assigns a number to each category (e.g., male=0, female=1) | Columns with only 2 categories, or categories with a natural order |
| **One-Hot Encoding** | Creates a new column for each category with 0/1 values | Columns with 3+ categories that have no natural order |

> **Why not just use label encoding for everything?** If we encode Embarked as C=0, Q=1, S=2, the model might think S > Q > C (i.e., that there's an order). One-hot encoding avoids this problem.

### 6a: Label Encoding for `Sex` (2 categories)

In [ ]:
# Sex has only 2 categories, so label encoding works well
print('Before encoding:')
print(df_clean['Sex'].value_counts())

# Map text to numbers
df_clean['Sex'] = df_clean['Sex'].map({'male': 0, 'female': 1})

print('\nAfter encoding:')
print(df_clean['Sex'].value_counts())

### 6b: One-Hot Encoding for `Embarked` (3 categories)

In [ ]:
# Embarked has 3 categories (C, Q, S) with no natural order → use one-hot encoding
print('Before encoding:')
print(df_clean['Embarked'].value_counts())

# pd.get_dummies() creates one new column per category
# drop_first=True removes one column to avoid redundancy
#   (if it's NOT Embarked_Q and NOT Embarked_S, it must be C)
df_clean = pd.get_dummies(df_clean, columns=['Embarked'], drop_first=True)

print('\nAfter encoding — new columns added:')
df_clean.head()

In [ ]:
# Check: no more 'object' columns!
print('Data types after encoding:')
print(df_clean.dtypes)
print(f'\nAll columns are now numerical: {all(df_clean.dtypes != "object")}')

Now every column is numerical and ready for machine learning algorithms.

---
## 7. Detect and Handle Outliers

Outliers are extreme values that are far from the rest of the data. They can distort statistics and model training.

We'll use two common detection methods:
1. **Visual inspection** with box plots
2. **IQR (Interquartile Range) method** — a standard statistical approach

### 7a: Visualize with Box Plots

In [ ]:
# Box plots for Age and Fare
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].boxplot(df_clean['Age'], vert=False)
axes[0].set_xlabel('Age')
axes[0].set_title('Box Plot of Age')

axes[1].boxplot(df_clean['Fare'], vert=False)
axes[1].set_xlabel('Fare')
axes[1].set_title('Box Plot of Fare')

plt.tight_layout()
plt.show()

The dots beyond the "whiskers" are potential outliers. Notice that `Fare` has some extreme values on the right side.

### 7b: IQR Method for Outlier Detection

The IQR method defines outliers as values that fall below `Q1 - 1.5 × IQR` or above `Q3 + 1.5 × IQR`.

- **Q1** = 25th percentile
- **Q3** = 75th percentile
- **IQR** = Q3 − Q1

In [ ]:
def detect_outliers_iqr(data, column):
    """Detect outliers using the IQR method. Returns a boolean mask."""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = (data[column] < lower_bound) | (data[column] > upper_bound)

    print(f'{column}:')
    print(f'  Q1 = {Q1:.2f}, Q3 = {Q3:.2f}, IQR = {IQR:.2f}')
    print(f'  Lower bound = {lower_bound:.2f}, Upper bound = {upper_bound:.2f}')
    print(f'  Number of outliers: {outliers.sum()} ({outliers.sum()/len(data)*100:.1f}%)')
    return outliers

age_outliers = detect_outliers_iqr(df_clean, 'Age')
print()
fare_outliers = detect_outliers_iqr(df_clean, 'Fare')

### 7c: Remove Outliers from Age

We'll remove Age outliers since they are a small percentage. We'll leave Fare outliers for now. Why? In the Titanic context, expensive first-class tickets are legitimate data points, not errors.

> **Important:** Always think about the *domain context* before removing outliers. Not every extreme value is an error!

In [ ]:
# Remove Age outliers
print(f'Rows before removing Age outliers: {len(df_clean)}')
df_clean = df_clean[~age_outliers]
print(f'Rows after:                        {len(df_clean)}')

---
## 8. Feature Scaling (Normalization)

Many machine learning algorithms work better when numerical features are on the **same scale**.

| Method | Formula | Range | When to use |
|--------|---------|-------|-------------|
| **Min-Max Scaling** | (x - min) / (max - min) | [0, 1] | When you want bounded values |
| **Standardization** | (x - mean) / std | ~[-3, 3] | When data is roughly normal |

We'll demonstrate Min-Max scaling here.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Get the numerical columns to scale
# We exclude 'Survived' (target) and the one-hot columns (already 0/1)
num_cols_to_scale = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

# Make a copy so we don't modify our cleaned data
df_scaled = df_clean.copy()

# Apply Min-Max scaling
scaler = MinMaxScaler()
df_scaled[num_cols_to_scale] = scaler.fit_transform(df_scaled[num_cols_to_scale])

print('Scaled data (first 5 rows):')
df_scaled.head()

In [ ]:
# Verify: all scaled columns should now be between 0 and 1
df_scaled[num_cols_to_scale].describe().round(2)

---
## 9. Summary: Our Data Cleaning Pipeline

Here's what we did, step by step:

| Step | What we did | Why |
|------|-------------|-----|
| 1 | Loaded data, checked `.head()`, `.shape` | Understand the data |
| 2 | Used `.info()` to check types and non-null counts | Find missing values and type issues |
| 3 | Used `.describe()` for summary statistics | Spot anomalies, understand distributions |
| 4 | Checked for duplicates with `.duplicated()` | Avoid double-counting |
| 5a | Dropped columns: Cabin (77% missing), Name, Ticket, PassengerId | Remove unusable or irrelevant features |
| 5b | Dropped 2 rows with missing Embarked | Very few missing — safe to drop |
| 5c | Filled missing Age with the median | Preserve data; median is robust to outliers |
| 6 | Encoded `Sex` (label) and `Embarked` (one-hot) | Convert text to numbers so algorithms can use them |
| 7 | Detected outliers with IQR; removed Age outliers | Clean extreme values (with domain judgment) |
| 8 | Applied Min-Max scaling to numerical features | Put features on same scale for modeling |